In [1]:
import rebound
import numpy as np
from numpy.linalg import eig
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from celmech.nbody_simulation_utilities import set_time_step,align_simulation
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
from celmech.secular import LaplaceLagrangeSystem
from celmech.poincare import Poincare
from celmech.disturbing_function import laplace_b as b
from scipy.optimize import fsolve
import cmath

In [9]:
def make_sim(alpha, mass, ecc, pom):
    mu1, mu2, mu3 = mass
    alpha12, alpha23 = alpha
    P1, P2, P3 = alpha12**(3/2), 1, 1/alpha23**(3/2)
    ecc1, ecc2, ecc3 = ecc
    pomega1, pomega2, pomega3 = pom
    
    # start simulation
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')

    # add star, planet 1, planet 2
    sim.add(m=1.)
    sim.add(m=mu1, P=P1, e=ecc1, pomega=pomega1)
    sim.add(m=mu2, P=P2, e=ecc2, pomega=pomega2,f=0.5)
    sim.add(m=mu3, P=P3, e=ecc3, pomega=pomega3,f=0.8)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.ri_whfast.safe_mode = 0
    sim.dt = sim.particles[1].P/12
    sim.collision = "direct"
    
    return sim

In [ ]:
def run(sim, simplified):
    ps = sim.particles
    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    vals,T = np.linalg.eigh(M) # T returned by eigh is R^T in paper. Could define R = T.T
    Mdiag = T.T @ lsys.Neccentricity_matrix @ T
    
    ec13 = 1-ps[1].a/ps[3].a
    m1, m2, m3 = ps[1].m, ps[2].m, ps[3].m
    m_tot = m1 + m2 + m3
    mu1, mu2, mu3 = m1/m_tot, m2/m_tot, m3/m_tot
    Tsec = 2*np.pi/(1/2*m_tot/ps[0].m/ec13**2)*ps[1].P
    
    alpha12, alpha23 = ps[1].a/ps[2].a, ps[2].a/ps[3].a
    alpha13 = alpha12*alpha23
    
    if simplified == 'yes':
        ec12, ec23, ec13 = 1-alpha12, 1-alpha23, 1-alpha13
    elif simplified == 'no':
        ec12 = alpha12**(-1/4)*alpha23**(3/4)*alpha23**(-1/8)*(1-alpha12)
        ec23 = alpha23**(-1/2)*alpha12**(1/8)*(1-alpha23)
        ec13 = alpha13**(-1/2)*(1-alpha13)
    
    Delta = (ec12 - ec23)/ec13
    a = (1-Delta)**3*(3+Delta)
    b = (1+Delta)**3*(3-Delta)
    A = mu1*a/(mu1*a+mu3*b)
    B = mu3*b/(mu1*a+mu3*b)
    
    Nout = 1000
    times = np.linspace(0,2*Tsec, Nout)
    Gx, Gy = np.zeros((Nout,3)), np.zeros((Nout,3))
    S1a, S2a, S3a = np.zeros(Nout), np.zeros(Nout), np.zeros(Nout)
    for i, time in enumerate(times):
        sim.integrate(time)    
        if simplified == 'yes':
            e1x, e2x, e3x = [p.e*np.cos(p.pomega) for p in ps[1:]]
            e1y, e2y, e3y = [p.e*np.sin(p.pomega) for p in ps[1:]]
            Gx[i] = [np.sqrt(p.m)*p.e*np.cos(-p.pomega) for p in ps[1:]]
            Gy[i] = [np.sqrt(p.m)*p.e*np.sin(-p.pomega) for p in ps[1:]]
        elif simplified == 'no':
            e1x, e2x, e3x = [p.e*np.cos(p.pomega)*(p.a)**(1/4) for p in ps[1:]]
            e1y, e2y, e3y = [p.e*np.sin(p.pomega)*(p.a)**(1/4) for p in ps[1:]]
            Gx[i] = [np.sqrt(2*p.m*np.sqrt(sim.G*ps[0].m*p.a)*(1-np.sqrt(1-p.e**2)))*np.cos(-p.pomega) for p in ps[1:]]
            Gy[i] = [np.sqrt(2*p.m*np.sqrt(sim.G*ps[0].m*p.a)*(1-np.sqrt(1-p.e**2)))*np.sin(-p.pomega) for p in ps[1:]]

        S1a[i] = np.sqrt((B*(e3x-e2x)-A*(e2x-e1x))**2 + (B*(e3y-e2y)-A*(e2y-e1y))**2)
        
        c_e12 = (mu3-mu1*mu2)*b+(mu1-mu2*mu3)*a + 16*mu2*Delta*mu1
        c_e23 = (mu3-mu1*mu2)*b+(mu1-mu2*mu3)*a - 16*mu2*Delta*mu3
        c1, c2, c3 = -c_e12, (c_e12-c_e23), c_e23
        S2a[i] = np.sqrt((c1*e1x+c2*e2x+c3*e3x)**2 + (c1*e1y+c2*e2y+c3*e3y)**2)
#         S2a[i] = np.sqrt((e3x-e1x)**2+(e3y-e1y)**2)
        S3a[i] = np.sqrt((mu1*e1x+mu2*e2x+mu3*e3x)**2+(mu1*e1y+mu2*e2y+mu3*e3y)**2)
    
    Sx = T.T @ Gx.T
    Sy = T.T @ Gy.T
       
    G1, G2, G3 = np.sqrt(Gx**2 + Gy**2).T
    S1, S2, S3 = np.sqrt(Sx**2 + Sy**2)
    
    return times, G1, G2, G3, S1, S2, S3, S1a, S2a, S3a

In [18]:
def calcMode(sim, simplified, mode):
    ps = sim.particles
    m_tot = ps[1].m + ps[2].m + ps[3].m
    
    m1, m2, m3 = ps[1].m, ps[2].m, ps[3].m
    mu1, mu2, mu3 = m1/m_tot, m2/m_tot, m3/m_tot
    alpha12, alpha23 = ps[1].a/ps[2].a, ps[2].a/ps[3].a
    alpha13 = alpha12*alpha23
    
    if simplified == 'yes':
        ec12, ec23, ec13 = 1-alpha12, 1-alpha23, 1-alpha13
    elif simplified == 'no':
        ec12 = alpha12**(-1/4)*alpha23**(3/4)*alpha23**(-1/8)*(1-alpha12)
        ec23 = alpha23**(-1/2)*alpha12**(1/8)*(1-alpha23)
        ec13 = alpha13**(-1/2)*(1-alpha13)
    
    Delta = (ec12 - ec23)/ec13
    a = (1-Delta)**3*(3+Delta)
    b = (1+Delta)**3*(3-Delta)
    
    if mode == 1:
        c_e12 = -mu1*((mu1+mu3)*a+16*mu3*Delta-mu2*(mu1+mu3)*b+16*mu2*mu3*Delta) + 16*mu1*mu3*Delta
        c_e23 = mu3*((mu1+mu3)*a+16*mu3*Delta-mu2*(mu1+mu3)*b+16*mu2*mu3*Delta) + 16*mu1*mu3*Delta
#         print(16*mu2*Delta*mu3**2/(mu3*b))
#         c_e12 = -mu1*a
#         c_e23 = mu3*b
        c1, c2, c3 = -c_e12, (c_e12-c_e23), c_e23
    elif mode == 2:
        c1, c2, c3 = -1, 0, 1
        c_e12 = (mu3-mu1*mu2)*b+(mu1-mu2*mu3)*a + 16*mu2*Delta*mu1
        c_e23 = (mu3-mu1*mu2)*b+(mu1-mu2*mu3)*a - 16*mu2*Delta*mu3
        print(16*mu2*Delta*mu3/((mu3-mu1*mu2)*b+(mu1-mu2*mu3)*a))
        c1, c2, c3 = -c_e12, (c_e12-c_e23), c_e23
    elif mode == 3:
        c1, c2, c3 = m1, m2, m3
    
    if simplified == 'yes':
        return 1/np.sqrt(c1**2+c2**2+c3**2) * np.array([c1, c2, c3])
    elif simplified == 'no':
#         print((ps[1].a)**(1/4), (ps[2].a)**(1/4), (ps[3].a)**(1/4))
        c1, c2, c3 = c1*(ps[1].a)**(1/4), c2*(ps[2].a)**(1/4), c3*(ps[3].a)**(1/4)
        return 1/np.sqrt(c1**2+c2**2+c3**2) * np.array([c1, c2, c3])

In [19]:
def modeDistance(sim, simplified, mode):
    ps = sim.particles  
    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    vals,T = np.linalg.eigh(M)
    m1, m2, m3 = ps[1].m, ps[2].m, ps[3].m
    
    for i in range(3):
        if simplified == 'yes':
            T[0][i] = T[0][i]*np.sqrt(m1)
            T[1][i] = T[1][i]*np.sqrt(m2)
            T[2][i] = T[2][i]*np.sqrt(m3)
        elif simplified == 'no':    
            T[0][i] = T[0][i]*np.sqrt(m1)*(sim.G*ps[0].m*ps[1].a)**(1/4)
            T[1][i] = T[1][i]*np.sqrt(m2)*(sim.G*ps[0].m*ps[2].a)**(1/4)
            T[2][i] = T[2][i]*np.sqrt(m3)*(sim.G*ps[0].m*ps[3].a)**(1/4)
        
    exp_theo = (T.T)[mode-1]
    vec_theo = 1/np.linalg.norm(exp_theo) * exp_theo
    vec_approx = calcMode(sim, simplified, mode)
    
#     print(vec_theo)
#     print(vec_approx)
    
    dist = min([np.linalg.norm(vec_theo-vec_approx),np.linalg.norm(vec_theo+vec_approx)])
    
    return dist

In [ ]:
def amp(x):
    return (max(x)-min(x))/(2*np.mean(x))

In [ ]:
def calcPsi(sim, simplified):
    ps = sim.particles
    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    vals,T = np.linalg.eigh(M) # T returned by eigh is R^T in paper. Could define R = T.T
    Mdiag = T.T @ lsys.Neccentricity_matrix @ T
    
    m1, m2, m3 = ps[1].m, ps[2].m, ps[3].m
    m_tot = m1 + m2 + m3
    mu1, mu2, mu3 = m1/m_tot, m2/m_tot, m3/m_tot
    alpha12, alpha23 = ps[1].a/ps[2].a, ps[2].a/ps[3].a
    alpha13 = alpha12*alpha23
    
    if simplified == 'yes':
        ec12, ec23, ec13 = 1-alpha12, 1-alpha23, 1-alpha13
    elif simplified == 'no':
        ec12 = alpha12**(-1/4)*alpha23**(3/4)*alpha23**(-1/8)*(1-alpha12)
        ec23 = alpha23**(-1/2)*alpha12**(1/8)*(1-alpha23)
        ec13 = alpha13**(-1/2)*(1-alpha13)
    
    R2 = np.array([[np.sqrt(m1*m2/(m1+m3)/m_tot), -np.sqrt((m1+m3)/m_tot), np.sqrt(m2*m3/(m1+m3)/m_tot)],
                   [-np.sqrt(m3/(m1+m3)), 0, np.sqrt(m1/(m1+m3))],
                   [np.sqrt(m1/m_tot), np.sqrt(m2/m_tot), np.sqrt(m3/m_tot)]])
    
    Mdiagapprox = R2 @ lsys.Neccentricity_matrix @ R2.T
    psi_theo = 1/2*np.arctan(2*Mdiagapprox[1,0]/(Mdiagapprox[1,1]-Mdiagapprox[0,0]))
    
    Delta = (ec12 - ec23)/ec13
    mu = (mu3-mu1)/(mu1+mu3)
    a = (1-Delta)**3*(3+Delta)
    b = (1+Delta)**3*(3-Delta)
    
    psi_approx = -16*np.sqrt(mu1*mu2*mu3)*Delta/((3+6*Delta**2+8*mu*Delta-Delta**4) - mu2*(3+6*Delta**2-8*mu*Delta-Delta**4))
    return psi_theo, float(psi_approx), np.abs((psi_theo-psi_approx)/psi_theo)